<a href="https://colab.research.google.com/github/ZoliN/colab/blob/main/hdrnetgoogle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install virtualenv
!virtualenv env
!git clone https://github.com/google/hdrnet.git
%cd /content/hdrnet/hdrnet
#!source /content/env/bin/activate;pip install -r requirements.txt
#!source /content/env/bin/activate;pip install opencv-python
#!source /content/env/bin/activate;pip install pytest
#!wget https://developer.nvidia.com/compute/cuda/10.0/Prod/local_installers/cuda-repo-ubuntu1804-10-0-local-10.0.130-410.48_1.0-1_amd64
#!dpkg -i cuda-repo-ubuntu1804-10-0-local-10.0.130-410.48_1.0-1_amd64
#!apt-key add /var/cuda-repo-10-0-local-10.0.130-410.48/7fa2af80.pub
#!apt-get update -qq
#!apt-cache policy cuda
#!apt-get install cuda=10.0.130-1
#!apt-get install gcc-5 g++-5 -y -qq;
#!ln -s /usr/bin/gcc-5 /usr/local/cuda/bin/gcc;
#!ln -s /usr/bin/g++-5 /usr/local/cuda/bin/g++;
!apt-get install libmagic1
import os
import sys
os.environ['CUDA_HOME'] = '/usr/local/cuda'
os.environ['PATH'] += ':/content/hdrnet'
os.environ['LD_LIBRARY_PATH'] += ':/content/env/lib/python3.7/site-packages/tensorflow'
!echo $CUDA_HOME
!echo $PATH

In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


In [ ]:
%cd /content/
!rm -rf hdrnet
!git clone https://github.com/google/hdrnet.git

In [ ]:
!apt-get install clang-10 -y -qq;
!ln -sf /usr/bin/clang-10 /usr/local/cuda/bin/gcc;
!ln -sf /usr/bin/clang-10 /usr/local/cuda/bin/g++;

In [31]:
!mkdir -p /content/hdrnet/hdrnet/ops/third_party/
!ln -sf /usr/local/lib/python3.7/dist-packages/tensorflow/include/tensorflow/ /content/hdrnet/hdrnet/ops/third_party/
!mkdir -p /usr/local/lib/python3.7/dist-packages/tensorflow/include/third_party/gpus/cuda
!ln -s /usr/local/cuda/include /usr/local/lib/python3.7/dist-packages/tensorflow/include/third_party/gpus/cuda

In [ ]:
%cd /content/hdrnet/hdrnet
%env PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python
!make clean
!make

In [ ]:
!ls /usr/local/lib/python3.7/dist-packages/tensorflow/include/third_party/gpus/cuda/include

In [ ]:
!mkdir -p /usr/local/lib/python3.7/dist-packages/tensorflow/include/third_party/gpus/cuda
!ln -s /usr/local/cuda/include /usr/local/lib/python3.7/dist-packages/tensorflow/include/third_party/gpus/cuda

!mkdir -p /content/env2/lib/python3.7/dist-packages/tensorflow/include/third_party/gpus/cuda
!ln -s /usr/local/cuda/include /content/env2/lib/python3.7/dist-packages/tensorflow/include/third_party/gpus/cuda

In [34]:
!ls -l /usr/local/cuda/bin/gcc


ls: cannot access '/usr/local/cuda/bin/gcc': No such file or directory


In [51]:
#!virtualenv env2
#%cd /content/hdrnet/hdrnet
#!source /content/env2/bin/activate;pip install -r requirements.txt
!pip install tensorflow-gpu


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
TF_INC ?= `python -c 'import tensorflow as tf; print(tf.sysconfig.get_include())'`
# TF_INC ?= /usr/local/lib/python2.7/dist-packages/tensorflow/include
CUDA_HOME ?= /usr/local/cuda

SRC_DIR = ops

BUILD_DIR = build
LIB_DIR = lib

CC = c++ -std=c++17
NVCC = nvcc -std c++17
CFLAGS = -fPIC -I$(TF_INC) -O2 -D_GLIBCXX_USE_CXX11_ABI=0
LDFLAGS = -L$(CUDA_HOME)/lib64 -lcudart
NVFLAGS = -DGOOGLE_CUDA=1 -D__CUDA__ -x cu -Xcompiler -fPIC -I$(TF_INC) \
					-expt-relaxed-constexpr -Wno-deprecated-gpu-targets -ftz=true


SRC = bilateral_slice.cc bilateral_slice_apply.cc bilateral_slice_op.cc bilateral_slice_apply_op.cc
CUDA_SRC = bilateral_slice.cu.cc bilateral_slice_apply.cu.cc
CUDA_OBJ = $(addprefix $(BUILD_DIR)/,$(CUDA_SRC:.cc=.o))
SRCS = $(addprefix $(SRC_DIR)/, $(SRC))

all: $(LIB_DIR)/hdrnet_ops.so

# Main library
$(LIB_DIR)/hdrnet_ops.so: $(CUDA_OBJ) $(LIB_DIR) $(SRCS)
	$(CC) -shared -o $@ $(SRCS) $(CUDA_OBJ) $(CFLAGS) $(LDFLAGS) 

# Cuda kernels
$(BUILD_DIR)/%.o: $(SRC_DIR)/%.cc $(BUILD_DIR)
	$(NVCC) -c  $< -o $@ $(NVFLAGS)

$(BUILD_DIR):
	mkdir -p $@


$(LIB_DIR):
	mkdir -p $@

clean:
	rm -rf $(BUILD_DIR) $(LIB_DIR)


In [ ]:
  template <int I>
  using StorageT = internal_compressed_tuple::Storage<ElemT<I>, I>;

  template <int I>
  ElemT<I>&& get() && {
    return std::move(*this).StorageT<I>::get();
  }

  template <int I>
  constexpr const ElemT<I>&& get() const&& {
    return absl::move(*this).StorageT<I>::get();
  }